In [8]:
import csv
import os
import numpy as np
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import time
import dataprofiler as dp
from kaggle.api.kaggle_api_extended import KaggleApi
from dataprofiler.data_readers.csv_data import CSVData
api = KaggleApi()
api.authenticate()

In [2]:
# Profilers options (disable unused features to reduce compute time)
profiler_options = dp.ProfilerOptions()
# # Structured options
profiler_options.structured_options.multiprocess.is_enabled = False
profiler_options.structured_options.chi2_homogeneity.is_enabled = False
# # # Numerical (int, float)
# # # # Integer
profiler_options.structured_options.int.min.is_enabled = False
profiler_options.structured_options.int.max.is_enabled = False
profiler_options.structured_options.int.num_zeros.is_enabled = False
profiler_options.structured_options.int.num_negatives.is_enabled = False
profiler_options.structured_options.int.bias_correction.is_enabled = False
# # # # Float
profiler_options.structured_options.float.min.is_enabled = False
profiler_options.structured_options.float.max.is_enabled = False
profiler_options.structured_options.float.num_zeros.is_enabled = False
profiler_options.structured_options.float.num_negatives.is_enabled = False
profiler_options.structured_options.float.bias_correction.is_enabled = False
# # # Text
profiler_options.structured_options.text.vocab.is_enabled = False
profiler_options.structured_options.text.min.is_enabled = False
profiler_options.structured_options.text.max.is_enabled = False
profiler_options.structured_options.text.mode.is_enabled = False
profiler_options.structured_options.text.median.is_enabled = False
profiler_options.structured_options.text.sum.is_enabled = False
profiler_options.structured_options.text.variance.is_enabled = False
profiler_options.structured_options.text.skewness.is_enabled = False
profiler_options.structured_options.text.kurtosis.is_enabled = False
profiler_options.structured_options.text.histogram_and_quantiles.is_enabled = False
profiler_options.structured_options.text.median_abs_deviation.is_enabled = False
profiler_options.structured_options.text.bias_correction.is_enabled = False

In [4]:
df_exc = pl.read_csv('./attribute_extraction/0_100_exceptions_old.csv')

### 1. Encoding error
- Try 'utf-8', 'latin1', 'iso-8859-1', 'cp1252'
- Otherwise, auto-detect encoding, try to use that.
- Otherwise, fail.

*Affected*: 0, 1, 2, 3, 4, 10, 13, 14, 17, 18, 62, 67


In [27]:
i = 0
print(df_exc['file_path'][i])
print(df_exc['exception_str'][i])

../datasets\datasnaek_youtube-new\JPvideos.csv
Traceback (most recent call last):
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 501, in extract_file_attributes_to_df
    report, sample_size = create_profiler_report(data, profiler_options)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 321, in create_profiler_report
    sample_size = min(len(data.data), 500000)
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\data_readers\base_data.py", line 225, in __getattribute__
    returned = object.__getattribute__(self, name)
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\data_readers\base_data.py", line 70, in data
    self._load_data()
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\data_readers\structured_mixins.py", line 60, in _load_data
    self._data = self._load_data_from_file(self.input_file_path)
  File "C:\Use

In [13]:
data = dp.Data('./datasets/datasnaek_youtube-new/JPvideos.csv', options={"encoding": "iso-8859-1"})

data.file_encoding

'iso-8859-1'

In [127]:
# TODO: try known encodings otherwise fallback to auto-detecting the encoding an dusing that. Otherwise, fail.
import chardet
with open('./datasets/datasnaek_youtube-new/JPvideos.csv', 'rb') as file:
    print(chardet.detect(file.read(1000)))

{'encoding': 'utf-8', 'confidence': 0.99, 'language': ''}


### 2. Error tokenizing data
- "on_bad_lines": "skip" (https://stackoverflow.com/questions/18039057/python-pandas-error-tokenizing-data)

*Affected*: 4, 8

In [7]:
i = 4
print(df_exc['file_path'][i])
print(df_exc['exception_str'][i])

../datasets\rashikrahmanpritom_heart-attack-analysis-prediction-dataset\o2Saturation.csv
Traceback (most recent call last):
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 501, in extract_file_attributes_to_df
    report, sample_size = create_profiler_report(data, profiler_options)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 321, in create_profiler_report
    sample_size = min(len(data.data), 500000)
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\data_readers\base_data.py", line 225, in __getattribute__
    returned = object.__getattribute__(self, name)
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\data_readers\base_data.py", line 70, in data
    self._load_data()
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\data_readers\structured_mixins.py", line 60, in _load_data
    self._data = self._load_data_from

In [10]:
data = CSVData('./datasets/rashikrahmanpritom_heart-attack-analysis-prediction-dataset/o2Saturation.csv', options={'on_bad_lines': 'skip'})

# profile = dp.Profiler(data, samples_per_update=1000, options=profiler_options) 

# report = profile.report(report_options={"output_format":"serializable"})

ParserError: Error tokenizing data. C error: Expected 2 fields in line 1356, saw 3


In [12]:
df = pd.read_csv('./datasets/rashikrahmanpritom_heart-attack-analysis-prediction-dataset/o2Saturation.csv', on_bad_lines='skip')
data = dp.Data(data=df, data_type='csv', options={'on_bad_lines': 'skip'})

# profile = dp.Profiler(data, samples_per_update=1000, options=profiler_options) 

# report = profile.report(report_options={"output_format":"serializable"})
# report
data.file_encoding

'UTF-8'

In [ ]:
sniffer = csv.Sniffer()
data = open('./datasets/rashikrahmanpritom_heart-attack-analysis-prediction-dataset/o2Saturation.csv', "r").read(4096)
obj = sniffer.sniff(data)
obj.delimiter

In [11]:
i = 6
print(df_exc['file_path'][i])
print(df_exc['exception_str'][i])

../datasets\rounakbanik_the-movies-dataset\credits.csv
Traceback (most recent call last):
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 504, in extract_file_attributes_to_df
    attribute_dict = create_attributes_from_file_report(data, report)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 339, in create_attributes_from_file_report
    add_consistency_attributes(attributes_dict, feature_stats, global_stats)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 171, in add_consistency_attributes
    if feature_stats['data_type'] == 'integer':
TypeError: string indices must be integers



In [ ]:
data = dp.Data('./datasets/rounakbanik_the-movies-dataset/credits.csv')

profile = dp.Profiler(data, samples_per_update=100, profiler_options=profiler_options) 

report = profile.report(report_options={"output_format":"serializable"})

### 3. String Indices Must Be Integers
*Affected*: 6, 7, 9, 15, 16, 36, 66, 68

**Problem**: csv files got read as TextData which generated an Unstructured report (unstructured reports do not have 'data_type' field). Reason why they got read as TextData? Not sure, but profiler uses chardet and some proprietary methods for detecting encoding, delimiters and using those it decides whether file is indeed structured/csv. In this case, it decided it is not csv.

**Solution**: Read using CSVData object directly.

In [104]:
i = 9
print(df_exc['file_path'][i])
print(df_exc['exception_str'][i])

../datasets\gpreda_reddit-vaccine-myths\reddit_vm.csv
Traceback (most recent call last):
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 504, in extract_file_attributes_to_df
    attribute_dict = create_attributes_from_file_report(data, report)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 339, in create_attributes_from_file_report
    add_consistency_attributes(attributes_dict, feature_stats, global_stats)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 171, in add_consistency_attributes
    if feature_stats['data_type'] == 'integer':
TypeError: string indices must be integers



In [128]:
data = dp.Data('./datasets/gpreda_reddit-vaccine-myths/reddit_vm.csv', data_type='csv')

profile = dp.Profiler(data, options=profiler_options)

report = profile.report(report_options={"output_format":"serializable"})

ValueError: No data class types matched the input.

In [131]:
from dataprofiler.data_readers.csv_data import CSVData

data = CSVData('./datasets/gpreda_reddit-vaccine-myths/reddit_vm.csv')

profile = dp.Profiler(data, options=profiler_options)

report = profile.report(report_options={"output_format":"serializable"})

C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profiler_options.py:529: UserWarning: StructuredOptions.text.numeric_stats: The numeric stats are completely disabled.
  warnings.warn(


INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


100%|██████████| 8/8 [00:00<00:00, 39.11it/s]

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics... 



 25%|██▌       | 2/8 [00:01<00:04,  1.29it/s]

 38%|███▊      | 3/8 [00:02<00:03,  1.43it/s]

100%|██████████| 8/8 [00:10<00:00,  1.28s/it]


In [112]:
for feature_stats in report['data_stats']:
    if feature_stats['data_type'] == 'integer':
        print(1)

TypeError: string indices must be integers

In [134]:
data = CSVData('./text.xlsx', options={'encoding': 'latin1'})

profile = dp.Profiler(data, options=profiler_options)

report = profile.report(report_options={"output_format":"serializable"})

C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profiler_options.py:529: UserWarning: StructuredOptions.text.numeric_stats: The numeric stats are completely disabled.
  warnings.warn(


ParserError: Error tokenizing data. C error: Expected 2 fields in line 4, saw 3


### 4. Gini Impurity NoneType

*Affected*: 5, 12, 19, 20-35, 37-61, 63-65

**Solution**: check if 'gini_impurity' is not none in report

In [36]:
i = 5
print(df_exc['file_path'][i])
print(df_exc['exception_str'][i])

../datasets\uciml_breast-cancer-wisconsin-data\data.csv
Traceback (most recent call last):
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 504, in extract_file_attributes_to_df
    attribute_dict = create_attributes_from_file_report(data, report)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 341, in create_attributes_from_file_report
    add_accuracy_attributes(attributes_dict, feature_stats)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 299, in add_accuracy_attributes
    dict['avg_gini_impurity_categ_features'] += feature_stats['statistics']['gini_impurity']
TypeError: unsupported operand type(s) for +=: 'float' and 'NoneType'



### 5. predictions_list[index] = list(predictions[index][:sentence_length]) MemoryError

*Affected*: 11

In [103]:
i = 11
print(df_exc['file_path'][i])
print(df_exc['exception_str'][i])

../datasets\himanshupoddar_zomato-bangalore-restaurants\zomato.csv
Traceback (most recent call last):
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 501, in extract_file_attributes_to_df
    report, sample_size = create_profiler_report(data, profiler_options)
  File "D:\meng-code\kaggle_data\attribute_extraction\attribute_extraction.py", line 323, in create_profiler_report
    profile = Profiler(data, samples_per_update=sample_size, options=profiler_options)
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profile_builder.py", line 2804, in __new__
    return UnstructuredProfiler(
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profile_builder.py", line 1161, in __init__
    self.update_profile(data)
  File "C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profile_builder.py", line 908, in update_profile
    self._update_profile

### 6. dict['num_pii_entities'] += predicted_label_ratios[pii_label] * dict['n_rows'] TypeError: 'NoneType' object is not subscriptable

In [13]:

from dataprofiler.data_readers.csv_data import CSVData

data = pd.read_csv('./datasets/uciml_breast-cancer-wisconsin-data/data.csv', on_bad_lines='skip')

profile = dp.Profiler(data=dp.Data(data=data, data_type='csv'), options=profiler_options)

report = profile.report(report_options={"output_format":"serializable"})

C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profiler_options.py:529: UserWarning: StructuredOptions.text.numeric_stats: The numeric stats are completely disabled.
  warnings.warn(


INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


100%|██████████| 33/33 [00:00<00:00, 95.76it/s] 

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics... 



100%|██████████| 33/33 [00:16<00:00,  2.02it/s]


In [8]:
report['data_stats'][0]['statistics']['data_label_representation']

{'UNKNOWN': 0.0,
 'ADDRESS': 0.0,
 'BAN': 0.0,
 'CREDIT_CARD': 0.0,
 'DATE': 0.0,
 'TIME': 0.054481546572934976,
 'DATETIME': 0.0,
 'DRIVERS_LICENSE': 0.18101933216168717,
 'EMAIL_ADDRESS': 0.0,
 'UUID': 0.0,
 'HASH_OR_KEY': 0.0,
 'IPV4': 0.0,
 'IPV6': 0.0,
 'MAC_ADDRESS': 0.0,
 'PERSON': 0.0,
 'PHONE_NUMBER': 0.0,
 'SSN': 0.019332161687170474,
 'URL': 0.0,
 'US_STATE': 0.0,
 'INTEGER': 0.7451669595782073,
 'FLOAT': 0.0,
 'QUANTITY': 0.0,
 'ORDINAL': 0.0}

In [15]:
def hi():
    return 1

In [17]:
str(hi.__name__)

'hi'

Tests

In [22]:
df = pd.read_csv('./datasets/rounakbanik_the-movies-dataset/links.csv')

display(df)

data = dp.Data(data=df, data_type='csv')

profile = dp.Profiler(data, options=profiler_options)

report = profile.report(report_options={"output_format":"serializable"})


,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
45838,176269,6209470,439050.0
45839,176271,2028550,111109.0
45840,176273,303758,67758.0
45841,176275,8536,227506.0


C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profiler_options.py:529: UserWarning: StructuredOptions.text.numeric_stats: The numeric stats are completely disabled.
  warnings.warn(


INFO:DataProfiler.profilers.profile_builder: Finding the Null values in the columns... 


C:\Users\radue\AppData\Roaming\Python\Python39\site-packages\dataprofiler\profilers\profile_builder.py:2580: UserWarning: The data will be profiled with a sample size of 9168. All statistics will be based on this subsample and not the whole dataset.
  warnings.warn(
100%|██████████| 3/3 [00:00<00:00,  3.95it/s]

INFO:DataProfiler.profilers.profile_builder: Calculating the statistics... 



100%|██████████| 3/3 [00:23<00:00,  7.68s/it]


In [23]:
report

{'global_stats': {'samples_used': 9168,
  'column_count': 3,
  'row_count': 45843,
  'row_has_null_ratio': 0.00512652705061082,
  'row_is_null_ratio': 0.0,
  'unique_row_ratio': 1.0,
  'duplicate_row_count': 0,
  'file_type': 'csv',
  'encoding': 'UTF-8',
  'correlation_matrix': None,
  'chi2_matrix': None,
  'profile_schema': {'movieId': [0], 'imdbId': [1], 'tmdbId': [2]},
  'times': {'row_stats': 0.026999950408935547}},
 'data_stats': [{'column_name': 'movieId',
   'data_type': 'int',
   'data_label': 'TIME|INTEGER',
   'categorical': False,
   'order': 'random',
   'samples': ['70157', '146116', '3909', '166794', '147752'],
   'statistics': {'min': None,
    'max': None,
    'mode': [4848.6449999999995],
    'median': 108799.78159999989,
    'sum': 883585946.0,
    'mean': 96377.17561082024,
    'variance': 3309529409.190815,
    'stddev': 57528.5095338895,
    'skewness': -0.42414632630462756,
    'kurtosis': -1.1745093245411065,
    'histogram': {'bin_edges': [1.0,
      9278.7894